#Topic Modelling in Urdu by using LSA model
In this notebook we are going to implement LSA (Latent Semantic Analysis) model for urdu language.

##Mounting Google Drive with Collaboratory
If your dataset is present in google drive then do this step and join your google drive with collaboratory.

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


##Importing Dependencicies
We will import the required libraries which we are going to work in this notebook. Some of the libraries will be imported later as we build the model.

In [2]:
import pandas as pd
import numpy as np
import re

##Peeking the data

The we are going to reda csv file contaning more than 1 Million news from different sources related to different categories.

df.head() will print the firt 5 rows of the dataset.

[Dataset](https://data.mendeley.com/datasets/834vsxnb99/3)

In [3]:
df = pd.read_csv('/content/gdrive/MyDrive/Dataset(Colab)/UrduTopicModelling/urdu-news-dataset-1M.csv',encoding='utf-8')
df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Index,Headline,News Text,Category,Date,URL,Source,News length
0,0,عالمی بینک عسکریت پسندی سے متاثرہ خاندانوں کی ...,اسلام باد عالمی بینک خیبرپختونخوا کے قبائلی اض...,Business & Economics,12/6/2020,https://www.dawnnews.tv/news/1148499/,Dawn News,1854.0
1,1,مالی سال 2020 ریٹرن فائل کرنے والوں کی تعداد م...,اسلام باد فیڈرل بورڈ ریونیو ایف بی نے دسمبر کی...,Business & Economics,12/6/2020,https://www.dawnnews.tv/news/1148498/,Dawn News,2016.0
2,2,جاپان کو سندھ کے خصوصی اقتصادی زون میں سرمایہ ...,اسلام باد بورڈ انویسٹمنٹ بی او ئی کے چیئرمین ع...,Business & Economics,12/5/2020,https://www.dawnnews.tv/news/1148433/,Dawn News,2195.0
3,3,برامدات 767 فیصد بڑھ کر ارب 16 کروڑ ڈالر سے زائد,اسلام اباد پاکستان میں ماہ نومبر میں مسلسل تیس...,Business & Economics,12/5/2020,https://www.dawnnews.tv/news/1148430/,Dawn News,2349.0
4,4,کے الیکٹرک کو اضافی بجلی گیس کی فراہمی کے قانو...,اسلام باد نیشنل ٹرانسمیشن اینڈ ڈسپیچ کمپنی این...,Business & Economics,12/5/2020,https://www.dawnnews.tv/news/1148421/,Dawn News,2655.0


##Preprocessing of the Dataset

We will remove some extra coloumns first like date,source,url and some other columns too. Because we want only news foe now and all other things are extra for us.

There are 5 types of news in the data set. If you want, you can use only 1 or 2 types of news in the example. But for this Notebook i am going to take all types of news for Topic Modelling.

In [4]:
data_pre=df.drop(columns=['Index','Date','URL','Source','News length'])
data_pre.head()
# data_pre=data_pre.loc[df['Category'] == 'Business & Economics']

,Headline,News Text,Category
0,عالمی بینک عسکریت پسندی سے متاثرہ خاندانوں کی ...,اسلام باد عالمی بینک خیبرپختونخوا کے قبائلی اض...,Business & Economics
1,مالی سال 2020 ریٹرن فائل کرنے والوں کی تعداد م...,اسلام باد فیڈرل بورڈ ریونیو ایف بی نے دسمبر کی...,Business & Economics
2,جاپان کو سندھ کے خصوصی اقتصادی زون میں سرمایہ ...,اسلام باد بورڈ انویسٹمنٹ بی او ئی کے چیئرمین ع...,Business & Economics
3,برامدات 767 فیصد بڑھ کر ارب 16 کروڑ ڈالر سے زائد,اسلام اباد پاکستان میں ماہ نومبر میں مسلسل تیس...,Business & Economics
4,کے الیکٹرک کو اضافی بجلی گیس کی فراہمی کے قانو...,اسلام باد نیشنل ٹرانسمیشن اینڈ ڈسپیچ کمپنی این...,Business & Economics


We are going to remove all the extra ppunctuation marks, new line characters, tab space character and all other special characters from the text using <b>re</b> library

In [5]:
data_pre['News Text'] = \
data_pre['News Text'].map(lambda x: re.sub('[,\\.!?]', '', str(x)))
data_pre['News Text'].head()

0    اسلام باد عالمی بینک خیبرپختونخوا کے قبائلی اض...
1    اسلام باد فیڈرل بورڈ ریونیو ایف بی نے دسمبر کی...
2    اسلام باد بورڈ انویسٹمنٹ بی او ئی کے چیئرمین ع...
3    اسلام اباد پاکستان میں ماہ نومبر میں مسلسل تیس...
4    اسلام باد نیشنل ٹرانسمیشن اینڈ ڈسپیچ کمپنی این...
Name: News Text, dtype: object

##Stopwords
In nltk library there are many individual libraries for stopwords in different languages. If you found stopwords in urdu language in future then import nltk library and download them and use them. Otherwise I have make a stopword library in urdu by combining 3,4 different stopwords libraries. Now my stopword library has more than 900 stopwords in it. This is still not enough but if you want to add more stopwords in it then you can contribute to my github repository.

The stopword liibrary i have used is given in my repository which is on the link below:

[Stopwords Library](https://github.com/talhamasood0000/Topic_modelling_in_Urdu_LDA)

In [6]:
import nltk
stopwords=pd.read_csv('/content/gdrive/MyDrive/Dataset(Colab)/UrduTopicModelling/stopwords_updated.txt',names=['List'])
# stopwords['List']

stopwords_ur=[]
for li in stopwords['List']:
  stopwords_ur.append(li)

print(len(stopwords_ur))

907


##Gensim

Gensim is the library that mainly focuses on Topic Modelling. It has all the models implemented in it and you just have to use them by applying input and parameters

In [7]:
import gensim
from gensim.utils import simple_preprocess
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

The data variable is an arry in which each index is 1 news.

In [20]:
data = data_pre['News Text'].values.tolist()
print((data[2000]))

کراچی حکومت کی جانب سے سلیب کے ذریعے گیس کی قیمتوں میں رد بدل کو برامد کنندہ کمیونٹی کی جانب سے اجتماعی طور پر سراہا گیا ہےڈان اخبار کی رپورٹ کے مطابق ڈان سے بات کرتے ہوئے برامد کنندہ کمیونٹی کی جانب سے حکومتی فیصلے پر اطمینان کا اظہار کیا گیا کیونکہ گزشتہ برسوں سے کمیونٹی کی جانب سے گیس کی ریجنل قیمتیں برابر کرنے کے ساتھ ساتھ اسے مسابقتی بنانے کے لیے ٹیرف کم کرنے کا مطالبہ کیا جاتا رہا ہےتاہم برامد کنندہ کمیونٹی کے یہ دونوں مطالبات حکومت کے اعلان کے بعد پورے ہوتے نظر ارہے ہیں جس پر کمیونٹی کی جانب سے اطمینان کا اظہار کیا گیا ہےمزید پڑھیں متوسط طبقے کیلئے گیس کی قیمتوں میں 10 سے 20 فیصد تک اضافہاس حوالے سے پاکستان اپیرل فورم کے چیئرمین اور کراچی سے کپڑوں کے برامد کنندہ جاوید بلوانی کا کہنا تھا کہ ہم کئی برسوں سے زیرو ریٹڈ برمدی شعبوں کے لیے پانچ علیحدہ سلیبز کا مطالبہ کر رہے تھے اور ہمیں خوشی ہے کہ ہماری بات کو تسلیم کیا گیاخیال رہے کہ کراچی میں کاروبار کرنے والے اس اقدام سے کافی فکر مند تھے کیونکہ پنجاب اور کراچی کی صنعتوں کے درمیان گیس کی قیمتوں میں بڑا فرق کئی عرصے سے ان کے حق میں ت

##Building the LSA Model

We are going to first import TfidfVectorizer which will calculate Term Frequency-Inverse Term Frequency of each word and then store it in the vector.

We will also calculate SVD (Single Value Decomposition matrix ) using TruncatedSVD function imported from sklearn library.

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [10]:
vectorizer=TfidfVectorizer(stop_words=stopwords_ur,use_idf=True,encoding='utf-8')

X=vectorizer.fit_transform(data)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['آش', 'ئی', 'ئے', 'اٹھب', 'اپ', 'بة', 'بت', 'برہب', 'برہی', 'برہے', 'بضت', 'بپط', 'بگسیر', 'تبز', 'تی', 'تے', 'ثل', 'دکھب', 'دیکھ', 'ذر', 'ذوثبلا', 'ذکر', 'ذکرو', 'ذی', 'زیبد', 'ضبد', 'ضک', 'ضیک', 'طب', 'طجت', 'طی', 'طے', 'علاو', 'قطہ', 'لگ', 'هوک', 'هڑ', 'وخواى', 'وں', 'پب', 'پرا', 'پط', 'پوچھ', 'چب', 'چبئی', 'چبہ', 'چی', 'چے', 'ڈا', 'ڈلیب', 'ڈو', 'ڈھو', 'ڈی', 'ڈیں', 'کبل', 'کتہ', 'کور', 'کھول', 'کیو', 'گرو', 'ہب', 'یب', 'یقی'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [11]:
print(X.shape)
print(X[0])

(111862, 404539)
  (0, 184959)	0.03843740045977635
  (0, 97899)	0.052015575572325085
  (0, 64711)	0.061473076570725003
  (0, 248874)	0.025007515189306936
  (0, 123217)	0.05017721746926948
  (0, 217927)	0.0374251455343199
  (0, 35260)	0.0415518105653268
  (0, 315548)	0.04779274920138546
  (0, 187440)	0.07097739937178077
  (0, 279498)	0.06121200419165628
  (0, 49841)	0.05002972480660584
  (0, 183748)	0.05545919506011579
  (0, 394684)	0.046761781930338066
  (0, 147046)	0.05429641195053477
  (0, 76119)	0.03619015163568962
  (0, 217441)	0.03861790958313802
  (0, 174595)	0.08276859544393377
  (0, 83378)	0.04580766663985859
  (0, 112374)	0.03510142913668623
  (0, 187473)	0.05683213969185406
  (0, 157481)	0.0759899972551492
  (0, 184048)	0.0687638363779501
  (0, 355920)	0.05088194058128597
  (0, 306449)	0.04596060033135934
  (0, 120380)	0.021644372952391563
  :	:
  (0, 345759)	0.14683425745306156
  (0, 190211)	0.12785614260385797
  (0, 193294)	0.05030202521905833
  (0, 219608)	0.05104440791338

In [26]:
lsa=TruncatedSVD(n_components=5, n_iter=100)
lsa.fit(X)

TruncatedSVD(algorithm='randomized', n_components=5, n_iter=100,
             random_state=None, tol=0.0)

In [27]:
print(len(lsa.components_))
print(len(lsa.components_[0])) #lsa.components_ has  element and each element has further ____ elements in it

# import numpy as np
# num_array=np.array(lsa.components_)
# print(np.sum(num_array))

5
404539


##Results

This code will print all the results and find the topics per document

In [29]:
terms=vectorizer.get_feature_names() #it extracts all the actual words corresponding to the id
# print(len(terms))
for i,comp in enumerate(lsa.components_):
  termsInComp=zip(terms,comp) #map the 2 arrays(one array with TF-IDF Scores and other with corresponding word)
  sortedTerms=sorted(termsInComp, key=lambda x:x[1], reverse=True) [:50] #sorting to get maximum TF-IDF first and only taking first 10 values of it
  print("Concept:",i)
  for term in sortedTerms:
    print(term[0])
  print(" ")



Concept: 0
ٹیم
رنز
پاکستان
میچ
کرکٹ
ٹیسٹ
سیریز
فلم
خان
کپتان
پی
خلاف
حاصل
محمد
انگلینڈ
ٹی
بی
کھلاڑیوں
شکست
کپ
وکٹ
اننگز
سری
علی
کھلاڑی
جانب
بھارت
ورلڈ
پاکستانی
ڈے
قومی
ٹوئنٹی
ون
تین
لینڈ
بنا
بیٹنگ
لنکا
جیت
روپے
بورڈ
فائنل
سال
اسکور
احمد
میچز
نیوزی
بلے
مطابق
کھیل
 
Concept: 1
فلم
روپے
ڈالر
ارب
قیمت
خان
کروڑ
وڈ
مارکیٹ
ریلیز
سال
مطابق
کردار
فیصد
اداکار
پیسے
ہزار
جی
بینک
لاکھ
اضافہ
ٹیکس
کپور
اداکارہ
کام
حکومت
ایف
اضافے
کہانی
قیمتوں
فلموں
پی
ایم
جاری
فون
سونے
ایس
بولی
پڑھیں
پیش
نئی
سطح
بالی
اسٹاک
نظر
ئی
سلمان
انڈیکس
مالی
بی
 
Concept: 2
فلم
خان
وڈ
کردار
ریلیز
اداکار
کپور
کہانی
اداکارہ
فلموں
بالی
ووڈ
بولی
رخ
سلمان
ٹریلر
ہالی
رنز
گھروں
سینما
شاہ
کاسٹ
علی
بننے
کام
ایکشن
اداکاری
نمائش
نظر
زندگی
ادا
مرکزی
شوٹنگ
بھرپور
عامر
شادی
فلمیں
ایوارڈ
گانے
جوہر
زینت
ہدایت
دپیکا
ائیں
گھومتی
سنگھ
باکس
کامیڈی
بہترین
ممبئی
 
Concept: 3
رنز
روپے
فلم
اننگز
وکٹ
ڈالر
بنا
قیمت
مارکیٹ
اسکور
پیسے
وکٹوں
وکٹیں
سونے
بنائے
اضافے
بناکر
ہدف
اٹ
اضافہ
اوورز
ہزار
پویلین
نقصان
بیٹنگ
کروڑ
انڈیکس
پوائنٹس
حاصل
سنچری
قیمتوں
اسٹاک
